In [4]:
import json
from pprint import pprint
import h5py
import scipy.io
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from collections import Counter
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import csv
import ast
import random

Using TensorFlow backend.


In [2]:
def get_image(data,data_in,CATEGORIES):
    img = data[data_in].get('filename','')
    DATADIR = "sg_dataset"
    
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
        break
    return img_array

def get_image_new(img):
    DATADIR = "combined/"
    #Get Image from category
    img_array = cv2.imread(os.path.join(DATADIR,img), cv2.IMREAD_COLOR)
    return img_array

In [3]:
def get_object(data,data_in,rel_in):
    ob1 = data[data_in].get('relationships',)[rel_in].get('objects','')[0]
    ob2 = data[data_in].get('relationships',)[rel_in].get('objects','')[1]
    return ob1,ob2

In [4]:
def get_coordinate(data,data_in,ob1,ob2):
    y1 = data[data_in].get('objects','')[ob1].get('bbox','').get('y','')
    x1 = data[data_in].get('objects','')[ob1].get('bbox','').get('x','')
    w1 = data[data_in].get('objects','')[ob1].get('bbox','').get('w','')
    h1 = data[data_in].get('objects','')[ob1].get('bbox','').get('h','')

    y2 = data[data_in].get('objects','')[ob2].get('bbox','').get('y','')
    x2 = data[data_in].get('objects','')[ob2].get('bbox','').get('x','')
    w2 = data[data_in].get('objects','')[ob2].get('bbox','').get('w','')
    h2 = data[data_in].get('objects','')[ob2].get('bbox','').get('h','')
    return x1,y1,w1,h1,x2,y2,w2,h2

In [5]:
def UnionBox(x1,x2,y1,y2,h1,h2,w1,w2):
    ub_x = 0
    ub_y = 0
    ub_w = 0
    ub_h = 0

    if x1 > x2:
        ub_x = x2
    else:
        ub_x = x1

    if y1 > y2:
        ub_y = y2
    else:
        ub_y = y1

    if x1+w1 > x2+w2:
        ub_w = x1+w1-ub_x
    else:
        ub_w = x2+w2-ub_x

    if y1+h1 > y2+h2:
        ub_h = y1+h1-ub_y
    else:
        ub_h = y2+h2-ub_y
        
    return ub_x,ub_y,ub_w,ub_h

In [6]:
def crop_image(img_arr,ub_y,ub_x,ub_w,ub_h):
    #crop the image with the union box and return it
    #Use this for Union_WB_B method to black out the exterior
    crop_img = img_arr[int(ub_y):int(ub_y+ub_h), int(ub_x):int(ub_x+ub_w)].copy()
    return crop_img

In [7]:
def check_rel(rel):
    #predicats = ['above','behind','beneath','left of','right of',
    #             'below','has','hold','in','inside','in front of','near',
    #             'next to','on','on the left of','on the right of',
    #             'under','wear','at','has on','adjacent to','is behind the','is to left of',
    #             'is right of the','']
    
    spatial  = ['about','above','across','against','along','alongside','around'
                ,'at','atop','behind','beneath','below','beside','beyond'
                ,'close to','far from','front','inside','opposite','outside'
                ,'near','next','top','over','past','through','toward','within',
                'left','right','under']
    #Anything above 100 samples has been deleted
    spatial2 = ['above','at','behind','below','beside','front','in','near',
                'next','top','over','left','right','under']
    
    special = ['on','in','by','up']
    rel1 = rel
    #print(rel)
    for i in spatial2:
        if rel in special:
            return rel
        elif rel.find(i) != -1:
            #print(rel1)
            #print(i)
            rel = i
            return rel
        #elif i.find(rel) != -1:
            #rel = i
            #return rel
            #print("FAILURE ",rel1)
            
        #    print ("Found",rel)
    #rel = None
    #if rel not in spatial:
    #    rel = None
    rel = None
    return rel

#Objects aren't needed only saving relationships not training object classifier
def check_obj(obj1,obj2):
    objs = ['airplane','bag','ball','basket','bear','bed','bench',
            'bike','boat','bottle','bowl','box','building','bus',
            'bush','cabinet','camera','can','car','cart','cat','chair',
            'clock','coat','computer','cone','counter','cup','desk','dog',
            'elephant','engine','face','faucet','giraffe','glasses','grass',
            'hand','hat','helmet','horse','hydrant','jacket','jeans','keyboard',
            'kite','lamp','laptop','luggage','monitor','motorcycle','mountain',
            'mouse','oven','pants','paper','person','phone','pillow','pizza',
            'plane','plant','plate','post','pot','ramp','refrigerator','road',
            'roof','sand','shelf','shirt','shoe','shoes','shorts','sink','skateboard',
            'skis','sky','snowboard','sofa','stove','street',
            'suitcase','sunglasses','surfboard','table','tie',
            'tower','traffic light','train','trash can','tree',
            'trees','truck','umbrella','van','vase','watch','wheel']
    
    #if obj1 not in objs:
    #    obj1 = None
    
    #if obj2 not in objs:
    #    obj2 = None
    
    return obj1,obj2

In [8]:
def write_to_file(data,data_in,rel_in,crop_img,new_dir):
    
    ob1_name = data[data_in].get('relationships',)[rel_in].get('text','')[0]
    rel = data[data_in].get('relationships',)[rel_in].get('text','')[1]
    ob2_name = data[data_in].get('relationships',)[rel_in].get('text','')[2]
    rel = check_rel(rel)
    #ob1_name,ob2_name = check_obj(ob1_name,ob2_name)
    
    if rel != None:
        image_name = data[data_in].get('filename',)[:-4]+"_"+ob1_name+"_"+rel+"_"+ob2_name+"0"+".jpg"
        name = rel
        #Name directory after the label
        #name = rel
        dirName = new_dir+name
        try:
            # Create target Directory
            os.mkdir(dirName)
            #print("Directory " , dirName ,  " Created ") 
        except FileExistsError:
            pass

        filename = dirName+"/"+image_name.replace(" ", "")
        exists = False
        i = int(filename[len(filename)-5:-4])
        while exists != True:
            if (os.path.isfile(filename) != True):
                cv2.imwrite(filename,crop_img)
                exists = True
            else:
                i += 1
                filename = filename[:-5]+str(i)+".jpg"

In [9]:
def Union_WB_B_Seperate_New(height,width,x1,y1,h1,w1,x2,y2,h2,w2):
    
    img_array1 = img_array.copy()
    img_array2 = img_array.copy()
    
    #Sets to Green
    img_array1[0:height , 0:width] = (0,0,0)
    img_array1[int(y1):int(y1+h1), int(x1):int(x1+w1)] = (0,255,0)
    #Sets to Blue
    img_array2[0:height , 0:width] = (0,0,0)
    img_array2[int(y2):int(y2+h2), int(x2):int(x2+w2)] = (255,0,0)

    #return the blue and green images
    return img_array1,img_array2

def Union_WB_B_Seperate(img_array,x1,y1,h1,w1,x2,y2,h2,w2):
    
    img_array1 = img_array.copy()
    img_array2 = img_array.copy()
    
    #Sets to Green
    img_array1[0:img_array.shape[0] , 0:img_array.shape[1]] = (0,0,0)
    img_array1[int(y1):int(y1+h1), int(x1):int(x1+w1)] = (0,255,0)
    #Sets to Blue
    img_array2[0:img_array.shape[0] , 0:img_array.shape[1]] = (0,0,0)
    img_array2[int(y2):int(y2+h2), int(x2):int(x2+w2)] = (255,0,0)

    #return the blue and green images
    return img_array1,img_array2

def Union_WB_B(img_array,x1,y1,h1,w1,x2,y2,h2,w2):
    
    img_array[0:img_array.shape[0] , 0:img_array.shape[1]] = (0,0,0)
    img_array[int(y1):int(y1+h1), int(x1):int(x1+w1)] = (0,255,0)
    img_array[int(y2):int(y2+h2), int(x2):int(x2+w2)] = (255,0,0)
    
    return img_array

def Union_WB_B_R(img_array,x1,y1,h1,w1,x2,y2,h2,w2):
    
    img_array1 = img_array.copy()
    img_array2 = img_array.copy()
    img_array3 = img_array.copy()
    
    #Sets to Green
    img_array1[0:img_array.shape[0] , 0:img_array.shape[1]] = (0,0,0)
    img_array1[int(y1):int(y1+h1), int(x1):int(x1+w1)] = (0,255,0)
    
    #Sets to Blue
    img_array2[0:img_array.shape[0] , 0:img_array.shape[1]] = (0,0,0)
    img_array2[int(y2):int(y2+h2), int(x2):int(x2+w2)] = (255,0,0)
    
    #Sets the intersection to red
    # COORDINATES OF THE INTERSECTION BOX
    x3 = max(x1, x2)
    y3 = max(y1, y2)
    w3 = min(x1+w1, x2+w2)
    h3 = min(y1+h1, y2+h2)
    # AREA OF OVERLAP - Area where the boxes intersect
    width = (x3+w3 - x3)
    height = (y3+h3 - y3)
    img_array3[0:img_array.shape[0] , 0:img_array.shape[1]] = (0,0,0)
    # handle case where there is NO overlap
    if (width<0) or (height <0):
        print("No Overlap")
        #Do nothing
    else:    
        #Sets to Red
        img_array3[int(y3):int(y3+h3), int(x3):int(x3+w3)] = (0,0,255)
    
    #return the blue and green images
    return img_array1,img_array2,img_array3

def Union_WB(img_array,x1,y1,h1,w1,x2,y2,h2,w2):
    obj1 = img_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:].copy()
    obj2 = img_array[int(y2):int(y2+h2), int(x2):int(x2+w2),:].copy()

    #temp_array = img_array.copy()
    temp_array = np.zeros(img_array.shape, np.uint8)
    #temp_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:] = obj1
    #temp_array[:,:,:] = img_array[0:img_array.shape[0] , 0:img_array.shape[1],:]
    temp_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:] = img_array[int(y1):int(y1+h1), int(x1):int(x1+w1),:].copy()
    temp_array[int(y2):int(y2+h2), int(x2):int(x2+w2),:] = img_array[int(y2):int(y2+h2), int(x2):int(x2+w2),:].copy()

    return temp_array

In [10]:
#Old Method Irrelevant 
def execute_images(method,data,CATEGORIES):
    #Extracting the required data
    for i in range(len(data)):
        img_array = get_image(data,i,CATEGORIES)
        for j in range(len(data[i].get('relationships',))):
            try:
                obj1 , obj2 = get_object(data,i,j)
                x1,y1,w1,h1,x2,y2,w2,h2 = get_coordinate(data,i,obj1,obj2)
                ub_x,ub_y,ub_w,ub_h = UnionBox(x1,x2,y1,y2,h1,h2,w1,w2)
                
                if method is "Union_WB_B":
                    img_array1 = Union_WB_B(img_array.copy(),x1,y1,h1,w1,x2,y2,h2,w2)
                    crop_img = crop_image(img_array1,ub_y,ub_x,ub_w,ub_h)
                    crop_img = cv2.resize(crop_img, (224, 224))
                    write_to_file(data,i,j,crop_img,"Union_WB_B/")
                elif method is "Union":
                    crop_img = crop_image(img_array,ub_y,ub_x,ub_w,ub_h)
                    crop_img = cv2.resize(crop_img, (224, 224))
                    write_to_file(data,i,j,crop_img,"Union/")
                elif method is "Union_WB":
                    img_array1 = Union_WB(img_array.copy(),x1,y1,h1,w1,x2,y2,h2,w2)
                    crop_img = crop_image(img_array1,ub_y,ub_x,ub_w,ub_h)
                    crop_img = cv2.resize(crop_img, (224, 224))
                    write_to_file(data,i,j,crop_img,"Union_WB/") 
                else:
                    print("ERROR")
                    break
            except:
                pass

In [11]:
def execute_images_tests(method,data):
    #Extracting the required data
    for i in range(len(data)):
        try:
            img_array = get_image_new(data[i][0])
            ob1 = ast.literal_eval(data[i][3])
            ob1 = [int(n) for n in ob1]

            x1 = abs(ob1[0])
            #print(x1)
            y1 = abs(ob1[1])
            w1 = abs(ob1[2])
            h1 = abs(ob1[3])

            ob2 = ast.literal_eval(data[i][4])
            ob2 = [int(n) for n in ob2]


            x2 = abs(ob2[0])
            y2 = abs(ob2[1])
            w2 = abs(ob2[2])
            h2 = abs(ob2[3])

            #x1,y1,w1,h1,x2,y2,w2,h2 = get_coordinate(data,i,obj1,obj2)
            ub_x,ub_y,ub_w,ub_h = UnionBox(x1,x2,y1,y2,h1,h2,w1,w2)
            #img_array1 = Union_WB_B(img_array.copy(),x1,y1,h1,w1,x2,y2,h2,w2)
            #img_array1,img_array2 = Union_WB_B(img_array.copy(),x1,y1,h1,w1,x2,y2,h2,w2)
            img_array1,img_array2 = Union_WB_B_Seperate(img_array.copy(),x1,y1,h1,w1,x2,y2,h2,w2)

            g = img_array1.copy()
            # set blue and red channels to 0
            g[:, :, 0] = 0
            g[:, :, 2] = 0

            b1,g1,r1 = cv2.split (g)

            b = img_array2.copy()
            # set blue and red channels to 0
            b[:, :, 1] = 0
            b[:, :, 2] = 0

            b2,g2,r2 = cv2.split (b)

            r = img_array1.copy()
            r[:, :, 0] = 0
            r[:, :, 1] = 0
            r[:, :, 2] = 0
            
            b3,g3,r3 = cv2.split(r)

            f
            crop_img = crop_image(new_img,ub_y,ub_x,ub_w,ub_h)
            crop_img = cv2.resize(crop_img.copy(), (32, 32))
            
            # create a flattened list of pixel values
            crop_img = [str(x) for x in crop_img.flatten()]
            x = data[i][5]
            x = ast.literal_eval(x)
            x = [n.strip() for n in x]

            if method is "Training":
                with open("Training_Union_WB_B_1_224.csv","a",newline="") as f1: 
                    cw = csv.writer(f1)
                    cw.writerow([x]+crop_img)
                f1.close()
            elif method is "Testing":
                with open("Testing_Union_WB_B_1_224.csv","a",newline="") as f2: 
                    cw = csv.writer(f2)
                    cw.writerow([x]+crop_img)
                f2.close()
            elif method is "Validation":
                with open("Validation_Union_WB_B_1_224.csv","a",newline="") as f3: 
                    cw = csv.writer(f3)
                    cw.writerow([x]+crop_img)
                f3.close()
            else:
                print("Error Need path") 
        except:
            print(ub_x,ub_y,ub_w,ub_h)
            print(x1,x2,y1,y2,w1,w2,h1,h2)
            print(data[i][0])
            pass

In [12]:
def execute_image_meta_data(method,data,number):
    train_path =  "TTV/Training_data_"+str(number)+".csv"
    test_path =  "TTV/Testing_data_"+str(number)+".csv"
    valid_path =  "TTV/Validation_data_"+str(number)+".csv"
    #Extracting the required data
    for i in range(len(data)):
        try:
            img_array = get_image_new(data[i][0])
            height = img_array.shape[0]
            width = img_array.shape[1]            
            #This is the Green BB data[i][3]
            #This is the Blue BB Data [i][4]
            
            ob1 = ast.literal_eval(data[i][3])
            ob1 = [int(n) for n in ob1]

            ob2 = ast.literal_eval(data[i][4])
            ob2 = [int(n) for n in ob2]

            x = data[i][5]
            x = ast.literal_eval(x)
            x = [n.strip() for n in x]
            if method is "Training":
                with open(train_path,"a",newline="") as f1: 
                    cw = csv.writer(f1)
                    cw.writerow([x]+[height,width]+[ob1]+[ob2])
                f1.close()
            elif method is "Testing":
                with open(test_path,"a",newline="") as f2: 
                    cw = csv.writer(f2)
                    cw.writerow([x]+[height,width]+[ob1]+[ob2])
                f2.close()
            elif method is "Validation":
                with open(valid_path,"a",newline="") as f3: 
                    cw = csv.writer(f3)
                    cw.writerow([x]+[height,width]+[ob1]+[ob2])
                f3.close()
            else:
                print("Error Need path") 
        except:
            print(data[i][0])
            pass

In [12]:
#from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import img_to_array

f = open("Training_Union_WB_B.csv", "r")
line = f.readline()
i = 0
images = []
IMAGE_DIMS = (224, 224, 3)
while i < 15:
    # attempt to read the next line of the CSV file
    line = f.readline()
    
    # check to see if the line is empty, indicating we have
    # reached the end of the file
    if line == "":
        # reset the file pointer to the beginning of the file
        # and re-read the line
        f.seek(0)
        line = f.readline()

        # if we are evaluating we should now break from our
        # loop to ensure we don't continue to fill up the
        # batch from samples at the beginning of the file
        if mode == "eval":
            break

    # extract the label and construct the image
    empty_string = ""
    empty_image = ""
    string_counter = 0
    comma_counter = 1
    for x in line.rstrip():
        if x == "\"":
            string_counter += 2
            print(x)
            print(empty_string)
        elif x == ",":
            string_counter += 1
            comma_counter += 1
            empty_string = empty_string+x
        elif x == "]":
            string_counter += 2
            empty_string = empty_string+x
            break
        else:
            string_counter += 1
            empty_string = empty_string+x
    
    x = ast.literal_eval(empty_string)
    #print(x)
    x = [n.strip() for n in x]
    print(x)
    #print(line.rstrip()[string_counter:])
    line = line.strip().split(",")

    image = np.array([int(x) for x in line[comma_counter:]], dtype="uint8")

    image = image.reshape((224, 224, 3))
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    #image = cv2.resize(image, (224, 224))
    #image = img_to_array(image)
    # update our corresponding batches lists
    images.append(image)
    i += 1
    

['on']
['over']
['on']
['over']
"

['in', 'above']
['on']
['on']
['behind']
"

['over', 'above']
['below']
"

['under', 'near']
['over']
['near']
['behind']
['on']


In [14]:
with open('sg_dataset/sg_train_annotations.json') as f:
    data = json.load(f)

In [15]:
with open('sg_dataset/sg_train_annotations.json') as f:
    data = json.load(f)

relationships = []
for i in range(len(data)):
    filename = data[i].get('filename','')
    for j in range(len(data[i].get('relationships',''))):
        relationship = data[i].get('relationships','')[j].get('text','')[1]
        relationships.append(relationship)
        
    

In [16]:
from collections import Counter

rel_counter = Counter(relationships)
print(rel_counter)

Counter({'on': 17033, 'has': 10874, 'above': 6145, 'next to': 5457, 'behind': 5207, 'wearing': 4804, 'under': 3691, 'in front of': 3557, 'near': 3290, 'in': 2915, 'below': 2130, 'beside': 1522, 'over': 1458, 'is on': 1333, 'wears': 1173, 'holding': 1148, 'by': 1081, 'is above': 809, 'is behind': 728, 'beneath': 727, 'on top of': 672, 'with': 590, 'attached to': 548, 'is in front of': 483, 'to the left of': 430, 'is next to': 415, 'is wearing': 396, 'is left of': 365, 'to the right of': 350, 'is right of': 325, 'holds': 308, 'is under': 304, 'have': 297, 'is below': 241, 'is beside': 232, 'are on': 211, 'sitting on': 205, 'left of': 205, 'standing on': 184, 'carrying': 176, 'besides': 175, 'is in': 170, 'inside': 169, 'riding': 168, 'against': 158, 'is near': 158, 'looking at': 155, 'right of': 145, 'are behind': 143, 'at': 142, 'touching': 141, 'covering': 135, 'underneath': 135, 'are above': 125, 'standing near': 102, 'around': 99, 'contains': 88, 'watching': 81, 'using': 81, 'is hold

In [36]:
new_vrd = []
for i in range(len(data)):
    filename = data[i].get('filename','')
    for j in range(len(data[i].get('relationships',''))):
        
        relationship = data[i].get('relationships','')[j].get('text','')[1]
        relationship = check_rel(relationship)
        
        if relationship != None:    
            
            ob1,ob2 = get_object(data,i,j)
            object_name_1 = data[i].get('relationships','')[j].get('text','')[0]
            object_name_2 = data[i].get('relationships','')[j].get('text','')[2]
            x1,y1,w1,h1,x2,y2,w2,h2 = get_coordinate(data,i,ob1,ob2)
            input_vrd = [filename,object_name_1,object_name_2,[x1,y1,w1,h1],[x2,y2,w2,h2],[relationship]]
            new_vrd.append(input_vrd)
        
        #print(object1+" "+relationship+" "+object2)
        #rel = data[data_in].get('relationships',)[rel_in].get('text','')[1]

In [37]:
print(len(new_vrd))
new_appeneded = []
preposition = []
old_vrd = new_vrd
lenght_of_list = len(new_vrd)
j = 0 
i = 0
while j < lenght_of_list:
    #print(i , j , lenght_of_list)
    if i > 5:
        i = j-5
    else:
        i = j
    while i < lenght_of_list:
        #print(new_vrd[j][0] ,new_vrd[i][0])
        if (i != j) and (new_vrd[j][0] == new_vrd[i][0]) and (new_vrd[j][1] == new_vrd[i][1]) and (new_vrd[j][2] == new_vrd[i][2]) and (new_vrd[j][3] == new_vrd[i][3]) and (new_vrd[j][4] == new_vrd[i][4]):
            set1 = set(new_vrd[j][5])
            set2 = set(new_vrd[i][5])
            print(set1 , set2 , i , j)
            new_appeneded = []
            preposition = []
            if set1 != set2 and ((not (set1.issubset(set2))) and (not (set2.issubset(set1)))):
                for k in range(len(new_vrd[j][5])):
                    preposition.append(new_vrd[j][5][k])
                for k1 in range(len(new_vrd[i][5])):
                    preposition.append(new_vrd[i][5][k1])
                    
                new_appeneded = [new_vrd[j][0],new_vrd[j][1],new_vrd[j][2],new_vrd[j][3],new_vrd[j][4],list(set(preposition))]
                new_vrd.remove(new_vrd[i])
                print(new_appeneded)
                new_vrd.insert(i,new_appeneded)
                #lenght_of_list -= 1
                j -= 3
                i -= 3
                if i < 0:
                    i = 0
                elif j < 0:
                    j = 0
            elif (set1 != set2) and (set1.issubset(set2)):
                new_vrd.remove(new_vrd[j])
                lenght_of_list -= 1
                j -= 3
                i -= 3
                if i < 0:
                    i = 0
                elif j < 0:
                    j = 0
            elif (set1 == set2) or (set2.issubset(set1)):
                #print("GETTING REMOVED 2",new_vrd[i])
                #print("GETTING REMOVED 21",new_vrd[j])
                new_vrd.remove(new_vrd[i])
                lenght_of_list -= 1
                j -= 3
                i -= 3
                if i < 0:
                    i = 0
                elif j < 0:
                    j = 0
                #i = j-1
                #i -= 1

        i += 1
    j += 1

73380
{'above'} {'in'} 21 20
['5979461295_1a8090a9ae_b.jpg', 'freezer', 'refrigerator', [140.0, 49.0, 565.0, 422.0], [138.0, 50.0, 564.0, 971.0], ['above', 'in']]
{'above'} {'above', 'in'} 21 20
{'above'} {'over'} 30 28
['5979461295_1a8090a9ae_b.jpg', 'sign', 'refrigerator', [434.0, 3.0, 73.0, 45.0], [138.0, 50.0, 564.0, 971.0], ['above', 'over']]
{'above'} {'above', 'over'} 30 28
{'near'} {'under'} 33 32
['1348163097_6d49676ae8_o.jpg', 'bag', 'sign', [163.0, 382.0, 14.0, 41.0], [26.0, 165.0, 40.0, 101.0], ['under', 'near']]
{'near'} {'under', 'near'} 33 32
{'on'} {'above'} 74 73
['4886337019_438df8bdd3_b.jpg', 'skateboard', 'ramp', [181.0, 731.0, 305.0, 155.0], [4.0, 443.0, 675.0, 580.0], ['above', 'on']]
{'on'} {'above', 'on'} 74 73
{'above'} {'in'} 77 74
['4886337019_438df8bdd3_b.jpg', 'man', 'skateboard', [183.0, 237.0, 377.0, 438.0], [181.0, 731.0, 305.0, 155.0], ['above', 'in']]
{'above'} {'above', 'in'} 77 74
{'on'} {'above'} 77 74
['4886337019_438df8bdd3_b.jpg', 'man', 'ramp', 

{'front'} {'at'} 592 591
['2226297726_92f46e3aa8_b.jpg', 'man', 'screen', [2.0, 99.0, 241.0, 667.0], [560.0, 29.0, 327.0, 290.0], ['at', 'front']]
{'front'} {'at', 'front'} 592 591
{'at', 'front'} {'beside'} 592 591
['2226297726_92f46e3aa8_b.jpg', 'man', 'screen', [2.0, 99.0, 241.0, 667.0], [560.0, 29.0, 327.0, 290.0], ['beside', 'at', 'front']]
{'at', 'front'} {'beside', 'at', 'front'} 592 591
{'beside', 'at', 'front'} {'at'} 592 591
{'next'} {'left'} 593 592
['2226297726_92f46e3aa8_b.jpg', 'man', 'bottle', [2.0, 99.0, 241.0, 667.0], [667.0, 350.0, 34.0, 110.0], ['next', 'left']]
{'next'} {'next', 'left'} 593 592
{'front'} {'below'} 596 593
['2226297726_92f46e3aa8_b.jpg', 'bottle', 'screen', [667.0, 350.0, 34.0, 110.0], [560.0, 29.0, 327.0, 290.0], ['below', 'front']]
{'front'} {'below', 'front'} 596 593
{'front'} {'next'} 594 593
['2226297726_92f46e3aa8_b.jpg', 'bottle', 'man', [667.0, 350.0, 34.0, 110.0], [2.0, 99.0, 241.0, 667.0], ['next', 'front']]
{'front'} {'next', 'front'} 594 

{'front'} {'on', 'front'} 969 968
{'in'} {'under'} 984 981
['5161016911_c84801d340_b.jpg', 'person', 'umbrella', [375.0, 356.0, 322.0, 233.0], [360.0, 160.0, 497.0, 429.0], ['in', 'under']]
{'in'} {'in', 'under'} 984 981
{'front'} {'behind'} 984 981
['5161016911_c84801d340_b.jpg', 'person', 'building', [375.0, 356.0, 322.0, 233.0], [0.0, 103.0, 352.0, 475.0], ['behind', 'front']]
{'front'} {'behind', 'front'} 984 981
{'next'} {'under'} 1001 1000
['4527308553_57e0e0348f_b.jpg', 'plane', 'spools', [257.0, 364.0, 522.0, 358.0], [26.0, 247.0, 472.0, 238.0], ['under', 'next']]
{'next'} {'under', 'next'} 1001 1000
{'in'} {'on'} 1026 1025
['7568880522_18abd0af4f_b.jpg', 'passenger', 'bus', [371.0, 266.0, 16.0, 42.0], [257.0, 123.0, 562.0, 463.0], ['in', 'on']]
{'in'} {'in', 'on'} 1026 1025
{'in'} {'in'} 1027 1026
{'in'} {'in'} 1027 1026
{'in'} {'on'} 1027 1026
['7568880522_18abd0af4f_b.jpg', 'driver', 'bus', [655.0, 244.0, 90.0, 86.0], [257.0, 123.0, 562.0, 463.0], ['in', 'on']]
{'in'} {'in',

{'top'} {'above'} 1749 1747
['3247900714_7b4c71b506_b.jpg', 'traffic light', 'pole', [718.0, 85.0, 182.0, 480.0], [791.0, 576.0, 50.0, 100.0], ['above', 'top']]
{'top'} {'above', 'top'} 1749 1747
{'in'} {'near'} 1752 1751
['3247900714_7b4c71b506_b.jpg', 'woman', 'pole', [190.0, 442.0, 50.0, 124.0], [227.0, 4.0, 47.0, 562.0], ['in', 'near']]
{'in'} {'in', 'near'} 1752 1751
{'in', 'near'} {'next'} 1753 1751
['3247900714_7b4c71b506_b.jpg', 'woman', 'pole', [190.0, 442.0, 50.0, 124.0], [227.0, 4.0, 47.0, 562.0], ['in', 'next', 'near']]
{'in', 'near'} {'in', 'next', 'near'} 1753 1751
{'under'} {'at'} 1762 1761
['8400328659_10d964f0fe_b.jpg', 'street', 'truck', [5.0, 481.0, 1016.0, 286.0], [121.0, 438.0, 501.0, 226.0], ['under', 'at']]
{'under'} {'under', 'at'} 1762 1761
{'near'} {'next'} 1783 1781
['9964390374_e9e6aca637_b.jpg', 'truck', 'man', [532.0, 465.0, 234.0, 252.0], [189.0, 482.0, 95.0, 216.0], ['next', 'near']]
{'near'} {'next', 'near'} 1783 1781
{'beside'} {'next'} 1783 1781
['996

{'behind'} {'by'} 2280 2279
['6190172387_0750f7bd22_b.jpg', 'man', 'boxes', [868.0, 179.0, 127.0, 266.0], [811.0, 424.0, 213.0, 163.0], ['by', 'behind']]
{'behind'} {'by', 'behind'} 2280 2279
{'front'} {'by'} 2282 2281
['6190172387_0750f7bd22_b.jpg', 'boxes', 'man', [811.0, 424.0, 213.0, 163.0], [868.0, 179.0, 127.0, 266.0], ['by', 'front']]
{'front'} {'by', 'front'} 2282 2281
{'at'} {'on'} 2283 2282
['6190172387_0750f7bd22_b.jpg', 'flag', 'building', [459.0, 5.0, 66.0, 64.0], [339.0, 1.0, 419.0, 349.0], ['on', 'at']]
{'at'} {'on', 'at'} 2283 2282
{'on'} {'at'} 2295 2294
['3366401243_c41703a681_b.jpg', 'clock', 'building', [787.0, 125.0, 37.0, 49.0], [0.0, 107.0, 1022.0, 319.0], ['on', 'at']]
{'on'} {'on', 'at'} 2295 2294
{'front'} {'next'} 2322 2321
['9299503837_8f28cc5fc8_b.jpg', 'lady', 'umbrella', [390.0, 597.0, 45.0, 84.0], [349.0, 564.0, 74.0, 61.0], ['next', 'front']]
{'front'} {'next', 'front'} 2322 2321
{'beside'} {'near'} 2331 2330
['2222074316_81a6d2bbc3_b.jpg', 'boy', 'man'

...

{'left'} {'next'} 63855 63854
['7062149521_0135937599_b.jpg', 'speaker 1', 'laptop', [108.0, 347.0, 100.0, 99.0], [69.0, 298.0, 341.0, 205.0], ['next', 'left']]
{'left'} {'next', 'left'} 63855 63854
{'right'} {'next'} 63859 63858
['7062149521_0135937599_b.jpg', 'refrigerator', 'table', [356.0, 441.0, 668.0, 241.0], [0.0, 434.0, 482.0, 223.0], ['right', 'next']]
{'right'} {'right', 'next'} 63859 63858
{'under'} {'at'} 63895 63894
['9149060968_9a9e26dc11_b.jpg', 'sand', 'man', [2.0, 423.0, 1021.0, 342.0], [139.0, 235.0, 510.0, 489.0], ['under', 'at']]
{'under'} {'under', 'at'} 63895 63894
{'on'} {'in'} 63908 63907
['5012012351_d903a3171b_b.jpg', 'guy', 'skateboard', [298.0, 301.0, 238.0, 245.0], [328.0, 444.0, 132.0, 98.0], ['in', 'on']]
{'on'} {'in', 'on'} 63908 63907
{'on'} {'in'} 63913 63912
['5012012351_d903a3171b_b.jpg', 'skateboard', 'wall', [328.0, 444.0, 132.0, 98.0], [68.0, 441.0, 608.0, 325.0], ['in', 'on']]
{'on'} {'in', 'on'} 63913 63912
{'on'} {'over'} 63914 63913
['50120123

{'below'} {'below'} 64424 64423
{'in'} {'next'} 64426 64424
['2828905413_30fe7ef5d8_o.jpg', 'leaves', 'stem', [3.0, 4.0, 1274.0, 955.0], [645.0, 712.0, 113.0, 118.0], ['in', 'next']]
{'in'} {'in', 'next'} 64426 64424
{'next'} {'in'} 64426 64424
['2828905413_30fe7ef5d8_o.jpg', 'leaves', 'brocolli', [3.0, 4.0, 1274.0, 955.0], [472.0, 372.0, 449.0, 362.0], ['in', 'next']]
{'next'} {'in', 'next'} 64426 64424
{'in', 'next'} {'in'} 64426 64424
{'in', 'next'} {'in'} 64427 64424
{'on'} {'above'} 64456 64455
['9321840072_2c68296247_b.jpg', 'food', 'table', [344.0, 315.0, 340.0, 340.0], [207.0, 340.0, 457.0, 425.0], ['above', 'on']]
{'on'} {'above', 'on'} 64456 64455
{'in'} {'over'} 64505 64500
['2247210571_808d7fc21e_b.jpg', 'boat', 'water', [371.0, 277.0, 360.0, 405.0], [6.0, 192.0, 1018.0, 489.0], ['over', 'in']]
{'in'} {'over', 'in'} 64505 64500
{'beside'} {'next'} 64505 64500
['2247210571_808d7fc21e_b.jpg', 'boat', 'boat', [371.0, 277.0, 360.0, 405.0], [682.0, 283.0, 342.0, 305.0], ['next',

{'under', 'below'} {'at'} 64871 64870
['3758547082_b8ce834a6e_b.jpg', 'pavement', 'sky', [0.0, 724.0, 681.0, 298.0], [5.0, 0.0, 677.0, 127.0], ['under', 'below', 'at']]
{'under', 'below'} {'under', 'below', 'at'} 64871 64870
{'over'} {'above'} 64872 64871
['3758547082_b8ce834a6e_b.jpg', 'sky', 'man', [5.0, 0.0, 677.0, 127.0], [344.0, 516.0, 270.0, 436.0], ['over', 'above']]
{'over'} {'over', 'above'} 64872 64871
{'on'} {'above'} 64924 64923
['972396591_7492461cfb_b.jpg', 'cross', 'tower', [184.0, 100.0, 28.0, 46.0], [31.0, 98.0, 333.0, 848.0], ['above', 'on']]
{'on'} {'above', 'on'} 64924 64923
{'front'} {'by'} 64934 64933
['972396591_7492461cfb_b.jpg', 'tree', 'church', [-1.0, 837.0, 149.0, 186.0], [3.0, 100.0, 677.0, 855.0], ['by', 'front']]
{'front'} {'by', 'front'} 64934 64933
{'above'} {'behind'} 64936 64935
['972396591_7492461cfb_b.jpg', 'sky', 'church', [2.0, 0.0, 679.0, 780.0], [3.0, 100.0, 677.0, 855.0], ['above', 'behind']]
{'above'} {'above', 'behind'} 64936 64935
{'in'} {'a

{'under'} {'front'} 65531 65530
['4207484255_f02d00b2e8_b.jpg', 'train', 'sky', [466.0, 383.0, 39.0, 55.0], [2.0, 0.0, 1016.0, 380.0], ['under', 'front']]
{'under'} {'under', 'front'} 65531 65530
{'over'} {'near'} 65581 65580
['9465172557_3f5ac3e1a0_b.jpg', 'ship', 'water', [151.0, 319.0, 579.0, 97.0], [4.0, 419.0, 1016.0, 285.0], ['over', 'near']]
{'over'} {'over', 'near'} 65581 65580
{'under'} {'below'} 65586 65585
['9465172557_3f5ac3e1a0_b.jpg', 'water', 'ship', [4.0, 419.0, 1016.0, 285.0], [151.0, 319.0, 579.0, 97.0], ['under', 'below']]
{'under'} {'under', 'below'} 65586 65585
{'under', 'below'} {'next'} 65586 65585
['9465172557_3f5ac3e1a0_b.jpg', 'water', 'ship', [4.0, 419.0, 1016.0, 285.0], [151.0, 319.0, 579.0, 97.0], ['under', 'below', 'next']]
{'under', 'below'} {'under', 'below', 'next'} 65586 65585
{'under', 'below', 'next'} {'near'} 65586 65585
['9465172557_3f5ac3e1a0_b.jpg', 'water', 'ship', [4.0, 419.0, 1016.0, 285.0], [151.0, 319.0, 579.0, 97.0], ['near', 'under', 'belo

{'in'} {'in'} 66421 66420
{'above'} {'on'} 66451 66450
['9358611438_30d0ee1a86_b.jpg', 'pillow', 'bed', [325.0, 605.0, 69.0, 50.0], [108.0, 598.0, 374.0, 295.0], ['above', 'on']]
{'above'} {'above', 'on'} 66451 66450
{'in'} {'in'} 66490 66489
{'by'} {'near'} 66501 66500
['5913822700_8e2dc2632e_b.jpg', 'table', 'pillow', [84.0, 424.0, 192.0, 88.0], [6.0, 517.0, 286.0, 166.0], ['by', 'near']]
{'by'} {'by', 'near'} 66501 66500
{'next'} {'right'} 66520 66519
['2184811526_749fbac815_b.jpg', 'man', 'wall', [464.0, 260.0, 106.0, 273.0], [1.0, 4.0, 222.0, 451.0], ['next', 'right']]
{'next'} {'next', 'right'} 66520 66519
{'beside'} {'near'} 66562 66561
['3129895560_c8132b20ed_o.jpg', 'board', 'time clock', [402.0, 132.0, 88.0, 98.0], [368.0, 198.0, 26.0, 39.0], ['beside', 'near']]
{'beside'} {'beside', 'near'} 66562 66561
{'front'} {'behind'} 66584 66583
['8081712501_49cfb2e305_b.jpg', 'man', 'railing', [237.0, 91.0, 115.0, 205.0], [-2.0, 131.0, 858.0, 210.0], ['behind', 'front']]
{'front'} {'b

In [32]:
for x in range(len(new_vrd)):
    for y in range(len(new_vrd)):
        if (new_vrd[x][0] == new_vrd[y][0]) and (new_vrd[x][1] == new_vrd[y][1]) and (new_vrd[x][2] == new_vrd[y][2]) and (x != y) and (new_vrd[x][3] == new_vrd[y][3]) and (new_vrd[x][4] == new_vrd[y][4]):
            print(new_vrd[x],x,y)

KeyboardInterrupt: 

In [38]:
print(len(new_vrd))

67313


In [39]:
import csv

with open("Multi_label_predictions_spatial2.csv","a",newline="") as f: 
    cw = csv.writer(f)
    cw.writerows(r+[""] for r in new_vrd)
f.close()
    
#print(set(new_vrd))

In [19]:
import csv
import ast
from collections import Counter

with open('Multi_label_predictions_spatial2.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)

all_labels = []
number_multi_labels = 0
singel_labels = 0 
combinations = []
for i in range(len(your_list)):
    x = your_list[i][5]
    x = ast.literal_eval(x)
    x = [n.strip() for n in x]
    if len(x) > 1:
        number_multi_labels += 1
        combinations.append(x)
        for j in x:
            singel_labels += 1
            all_labels.append(j)
    else:
        #print(x[0])
        #print(x)
        singel_labels += 1
        all_labels.append(x[0])

print(number_multi_labels)
print(singel_labels)
dictionary_labels = Counter(all_labels)
#dictionary_combinations = Counter(combinations)
        #print(your_list[i][5])


5261
90243


In [17]:
def stratified_sampling(your_list):
    print(len(your_list))
    #Store the counters of the labels
    train_labels = []
    test_labels =[]
    valid_labels = []
    #Store information of image
    train_info = []
    test_info = []
    valid_info = []

    #This should only iterate for multi labels first then
    #It will go through 
    all_labels = []
    multi_labels_count = 0
    single_labels_count = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    count6 = 0
    count7 = 0
    for i in range(len(your_list)):
        x = your_list[i][5]
        x = ast.literal_eval(x)
        x = [n.strip() for n in x]

        #print(Counter(test_labels).get(x[0]))
        #This is for multi labels
        if len(x) > 1:
            multi_labels_count += 1
            if (Counter(test_labels).get(x[0]) == None):
                count1 += 1
                test_info.append(your_list[i])
                for j in x:
                    test_labels.append(j)
            elif (Counter(valid_labels).get(x[0]) == None):
                count2 += 1
                valid_info.append(your_list[i])
                for j in x:
                    valid_labels.append(j)
            elif (Counter(train_labels).get(x[0]) == None):
                count3 += 1
                train_info.append(your_list[i])
                for j in x:
                    train_labels.append(j)
            else:
                if Counter(test_labels).get(x[0]) < Counter(valid_labels).get(x[0]):
                    count4 += 1
                    test_info.append(your_list[i])
                    for j in x:
                        test_labels.append(j)
                elif Counter(test_labels).get(x[0]) > Counter(valid_labels).get(x[0]):
                    count5 += 1
                    valid_info.append(your_list[i])
                    for j in x:
                        valid_labels.append(j)
                elif Counter(train_labels).get(x[0]) <= ((Counter(valid_labels).get(x[0]) + Counter(train_labels).get(x[0]) + Counter(test_labels).get(x[0])) * 0.60):
                    count6 += 1
                    train_info.append(your_list[i])
                    for j in x:
                        train_labels.append(j)
                elif Counter(test_labels).get(x[0]) == Counter(valid_labels).get(x[0]):
                    count4 += 1
                    test_info.append(your_list[i])
                    for j in x:
                        test_labels.append(j)
                else:
                    print(Counter(test_labels).get(x[0]))
                    print(Counter(valid_labels).get(x[0]))
                    print(Counter(train_labels).get(x[0]))
                    count7 += 1
        else:
            single_labels_count += 1
            if (Counter(test_labels).get(x[0]) == None):
                count1 += 1
                test_info.append(your_list[i])
                test_labels.append(x[0])

            elif (Counter(valid_labels).get(x[0]) == None):
                count2 += 1
                valid_info.append(your_list[i])
                valid_labels.append(x[0])
            elif (Counter(train_labels).get(x[0]) == None):
                count3 += 1
                train_info.append(your_list[i])
                train_labels.append(x[0])
            else:
                if Counter(test_labels).get(x[0]) < Counter(valid_labels).get(x[0]):
                    count4 += 1
                    test_info.append(your_list[i])
                    test_labels.append(x[0])
                elif Counter(test_labels).get(x[0]) > Counter(valid_labels).get(x[0]):
                    count5 += 1
                    valid_info.append(your_list[i])
                    valid_labels.append(x[0])
                elif Counter(train_labels).get(x[0]) <= ((Counter(valid_labels).get(x[0]) + Counter(train_labels).get(x[0]) + Counter(test_labels).get(x[0])) * 0.60):
                    count6 += 1
                    train_info.append(your_list[i])
                    train_labels.append(x[0])
                elif Counter(test_labels).get(x[0]) == Counter(valid_labels).get(x[0]):
                    count4 += 1
                    test_info.append(your_list[i])
                    test_labels.append(x[0])
                else:
                    print(Counter(test_labels).get(x[0]))
                    print(Counter(valid_labels).get(x[0]))
                    print(Counter(train_labels).get(x[0]))
                    count7 += 1

    return train_info,test_info,valid_info


In [18]:
def write_dataset_to_file(train_info,test_info,valid_info,number):
    
    execute_image_meta_data("Training",train_info,number)
    execute_image_meta_data("Testing",test_info,number)
    execute_image_meta_data("Validation",valid_info,number)

In [26]:
def create_10_random_datasets():
    #Read the list 
    with open('Multi_label_predictions_spatial2.csv', 'r') as f:
        reader = csv.reader(f)
        your_list = list(reader)

    #Creating 10 randomly distributed datasets having them stratified sampled.
    for i in range(10):
        random.shuffle(your_list)
        train_info,test_info,valid_info = stratified_sampling(your_list)
        write_dataset_to_file(train_info,test_info,valid_info,i)

                                                      

In [27]:
create_10_random_datasets()

84145
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
84145
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
84145
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
4392556686_44d71ff5a0_o.gif
43

In [28]:
def expand_multilabels(Data_type):
    for number in range(10):
        train_path =  "TTV/Training_data_"+str(number)+".csv"
        test_path =  "TTV/Testing_data_"+str(number)+".csv"
        valid_path =  "TTV/Validation_data_"+str(number)+".csv"
        
        train_path_softmax =  "SoftmaxTTV/Training_data_softmax_"+str(number)+".csv"
        test_path_softmax =  "SoftmaxTTV/Testing_data_softmax_"+str(number)+".csv"
        valid_path_softmax =  "SoftmaxTTV/Validation_softmax_data_"+str(number)+".csv"
        
        if Data_type == "Train":
            #Read the list 
            with open(train_path, 'r') as f1:
                reader = csv.reader(f1)
                your_list = list(reader)
            f1.close()
        elif Data_type == "Test":
            #Read the list 
            with open(test_path, 'r') as f2:
                reader = csv.reader(f2)
                your_list = list(reader)
            f2.close()
        elif Data_type == "Valid":
            #Read the list 
            with open(valid_path, 'r') as f3:
                reader = csv.reader(f3)
                your_list = list(reader)
            f3.close()
        else:
            print("ERROR")
            
        new_list = []
        for i in range(len(your_list)):
            x = your_list[i][0]
            x = ast.literal_eval(x)
            x = [n.strip() for n in x]
            if len(x) > 1:
                for j in range(len(x)):
                    temp_input = [x[j],your_list[i][1],your_list[i][2],your_list[i][3],your_list[i][4]]
                    new_list.append(temp_input)
            else:
                temp_input = [x[0],your_list[i][1],your_list[i][2],your_list[i][3],your_list[i][4]]
                new_list.append(temp_input)
                
        if Data_type == "Train":
            #Read the list 
            with open(train_path_softmax,"a",newline="") as f4: 
                cw = csv.writer(f4)
                cw.writerows(new_list)
            f4.close()
        elif Data_type == "Test":
            #Read the list 
            with open(test_path_softmax,"a",newline="") as f5: 
                cw = csv.writer(f5)
                cw.writerows(new_list)
            f5.close()
        elif Data_type == "Valid":
            #Read the list 
            with open(valid_path_softmax,"a",newline="") as f6: 
                cw = csv.writer(f6)
                cw.writerows(new_list)
            f6.close()
        else:
            print("ERROR")

In [29]:
expand_multilabels("Train")
expand_multilabels("Test")
expand_multilabels("Valid")

In [5]:
test_path =  "TTV/Testing_data_"+str(0)+".csv"

with open(test_path, 'r') as f1:
    reader = csv.reader(f1)
    your_list = list(reader)
f1.close()

In [12]:
for i in range(5):
    x = your_list[0][i]
    #x = ast.literal_eval(x)
    #x = [n.strip() for n in x]
    print(x)

['above']
800
523
[57, 311, 153, 119]
[190, 422, 22, 38]


In [32]:
f = open(test_path, "r")
line = f.readline()
new_line = line.rstrip()
print(new_line)

[


In [42]:
#for i in range(5):
x = your_list[0][3]
x = ast.literal_eval(x)
x = [int(n) for n in x]
print(x[0])

57
